<div style="width:100%; text-align:center; margin: 40px 0;">
    <h1 style="color: #003366; font-size: 40px; font-weight:bold; text-shadow: 2px 2px 5px grey;">
        Énergie Électrique en France
    </h1>
    <hr style="width: 50%; height: 3px; background-color: #003366; border: none;">
    <p style="font-style:italic; font-size:18px; color: #555;">
        Une exploration approfondie de l'énergie électrique en France
    </p>
</div>


In [4]:
# Data

URL = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/"

# Imports
import datetime
import json
import requests
import time

import pandas as pd
import plotly.express as px


In [2]:
def fetch_data_by_date(data:str, start:int, rows:int, date:str):
    """Fetch data from a dataset by date and offset
    
    Parameters
    ----------
    data : str
        Name of the dataset.
    start : int
        Offset.
    rows : int
        Number of rows.
    date : str
        Date of the data.

    Returns
    -------
    dict
        Dictionary containing the data. 
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "offset" : start,
        "rows": rows,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results', {})
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return 
    
def get_length_per_date(data:str, date:str):
    """Get the number of rows for a given date

    Parameters
    ----------
    data : str
        Name of the dataset.
    date : str
        Date of the data.

    Returns
    -------
    int
        Number of rows.
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('total_count')
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return
    
def insert_data_into_json(file_name: str, new_data: list):
    """Insère de nouvelles données dans un fichier JSON existant.

    Parameters
    ----------
    file_name : str
        Le nom du fichier JSON.
    new_data : list
        La liste des nouvelles données à insérer.
    """
    try:
        with open(file_name, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    existing_data.extend(new_data)

    with open(file_name, 'w') as file:
        json.dump(existing_data, file)

def get_date(data: str, first: bool=True) -> str:
    """Get the minimum or maximum date in a dataset

    Parameters
    ----------
    data : str
        Name of the dataset.
    first : bool, optional
        If True, get the minimum date, else get the maximum date. The default is True.
    
    Returns
    -------
    str
        Date.
    """
    date = "date" if first else "-date"
    
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "order_by": date,
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results')[0]['date']
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return

In [5]:

def get_data(from_data: str, json_name: str) -> None:
    """Get data from a dataset and insert it into a JSON file

    Parameters
    ----------
    from_data : str
        Name of the dataset.
    json_name : str
        Name of the JSON file.
    """
    step = 100 # Number of rows to be retrieved per request (API limit)
    start_date = get_date(from_data)
    end_date = get_date(from_data, first = False)
    current_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    
    while current_date <= end_date:
        formatted_date = str(current_date.strftime('%Y-%m-%d'))
        lines_per_date = get_length_per_date(from_data, str(formatted_date))
        print(formatted_date)
        
        for i in range(0, lines_per_date, step):
            rows = min(step, lines_per_date - i)
            try:
                data = fetch_data_by_date(from_data, i, rows, str(formatted_date))
                insert_data_into_json(json_name, data)
                    
            except Exception as e:
                print(f"Erreur lors de l'insertion des données pour la date {formatted_date}, offset {i}: {e}")
                print("Réessai dans 5 secondes...")
                time.sleep(5)  
                i -= step  
                continue
                
        current_date += datetime.timedelta(days=1)

get_data("eco2mix-national-tr", "national.json")

In [5]:
def transform_json_to_df(file_name:str):
    """Transform a Json File to a pandas DataFrame"""
    with open(file_name) as f:
        data = json.load(f)
    return pd.DataFrame(data)
    
df = transform_json_to_df('combined_data.json')
df

,perimetre,nature,date,heure,date_heure,consommation,prevision_j1,prevision_j,fioul,charbon,...,gaz_ccg,gaz_autres,hydraulique_fil_eau_eclusee,hydraulique_lacs,hydraulique_step_turbinage,bioenergies_dechets,bioenergies_biomasse,bioenergies_biogaz,stockage_batterie,destockage_batterie
0,France,Données temps réel,2023-11-01,00:00,2023-10-31T23:00:00+00:00,45304,45200,45500,223,6,...,887,0,5283,1659,872,73,356,245,-7,2
1,France,Données temps réel,2023-11-01,01:00,2023-11-01T00:00:00+00:00,40996,41000,41300,225,6,...,562,0,5239,1805,592,71,358,245,-11,4
2,France,Données temps réel,2023-11-01,04:30,2023-11-01T03:30:00+00:00,36058,35700,36200,230,6,...,215,0,4791,1401,27,89,355,245,-2,1
3,France,Données temps réel,2023-11-01,07:00,2023-11-01T06:00:00+00:00,38643,38700,38900,227,6,...,209,0,4790,1353,132,81,355,245,-1,0
4,France,Données temps réel,2023-11-01,07:30,2023-11-01T06:30:00+00:00,38925,38900,39000,224,6,...,148,0,4769,1705,327,82,355,245,-12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,France,Données temps réel,2023-11-30,12:00,2023-11-30T11:00:00+00:00,67609,68300,68000,186,587,...,5725,0,6313,4032,2960,71,506,246,-2,11
5756,France,Données temps réel,2023-11-30,14:15,2023-11-30T13:15:00+00:00,65158,65050,65200,187,350,...,5510,0,6325,3258,1986,74,507,246,-12,2
5757,France,Données temps réel,2023-11-30,16:45,2023-11-30T15:45:00+00:00,63335,64600,63650,397,541,...,5493,0,6115,3848,2710,76,510,246,-3,1
5758,France,Données temps réel,2023-11-30,17:00,2023-11-30T16:00:00+00:00,64063,65100,64000,315,596,...,5205,0,6186,3773,2512,75,512,246,-6,0


In [13]:
import matplotlib.pyplot as plt
from ipywidgets import interact

@interact(day=(1,30))
def line_chart_consumption_november(day: int = 1) -> None:
    """Generate an interactive line chart of electricity consumption in France for a specific day in November 2023.

    Parameters
    ----------
    day : int
        Day of November 2023.
    
    Returns
    -------
    None

    """
    df = transform_json_to_df('combined_data.json')
    df = df[df['date'] == f'2023-11-{day:02}']
    df = df.sort_values(by=['heure'])
    
    plt.figure(figsize=(13.5,3))
    plt.plot(df['heure'], df['consommation'])
    plt.xticks([f'{str(i).zfill(2)}:00' for i in range(24)], [f'{i:02}h' for i in range(24)])
    plt.xlim(left='00:00', right='23:45')
    plt.xlabel('Heure')
    plt.ylabel('Consommation')
    plt.title(f'Consommation électrique en France le {day:02} novembre 2023')

interactive(children=(IntSlider(value=1, description='day', max=30, min=1), Output()), _dom_classes=('widget-i…

In [11]:
import plotly.graph_objs as go
from ipywidgets import IntSlider, VBox, HBox
from IPython.display import display

# Créer le widget pour choisir le jour
day_slider = IntSlider(min=1, max=30, value=1, description='Jour', continuous_update=False)

# Initialiser la figure Plotly
fig = go.FigureWidget()

# Fonction de callback pour mettre à jour la figure
def update_figure(change):
    new_day = day_slider.value
    df = transform_json_to_df('combined_data.json')
    df = df[df['date'] == f'2023-11-{new_day:02}']
    df = df.sort_values(by=['heure'])
    with fig.batch_update():
        fig.data = []
        fig.add_scatter(x=df["heure"], y=df["consommation"], mode='lines')
        fig.layout.title = f'Consommation électrique en France le {new_day:02} novembre 2023'

# Ajouter le callback au widget
day_slider.observe(update_figure, names='value')

# Afficher le widget et la figure initiale
display(VBox([day_slider, fig]))

# Appeler manuellement pour initialiser la figure
update_figure(None)



…